# GenVeg Main Script
GenVeg is a generalized vegetation growth model that simulates growth due to
photosynthesis as well as processes that control vegetation distribution such
as mortality, senescence, dispersal, and colonization. The model utilizes a 
mixture of existing and new landlab components to setup the model, simulates
relevant vegetation community dynamics with the local environment, and analyzes
results of the simulation. 

## Main model control set up
### Import components

In [1]:
import numpy as np, pandas as pd
import os, tempfile
from pathlib import Path
from datetime import date
from matplotlib.pyplot import title, show, figure, plot, subplot, xlabel, ylabel
from landlab import RasterModelGrid, imshow_grid, load_params
from landlab.data_record import DataRecord
from landlab.components.genveg import VegParams, PlantGrowth, GenVeg
from landlab.io.netcdf import write_netcdf

### Define time controls

In [2]:
sim_start_date = np.datetime64('2010-01-01')
tot_sim_len_y = 30
veg_time_step_d = 1
env_time_step_d = 1
dt=np.timedelta64(min([veg_time_step_d,env_time_step_d]),'D')
run_length_d=365

### Define filenames for input files
##### *Note should be located in same folder as this .ipynb

In [3]:
VegInputFile = 'GenVeg_params_inputs_1col.xlsx'

### Read in parameter and data input files
Identify model processes to be included so all required parameters are defined and calculated

In [4]:
inp_file=Path(VegInputFile).absolute()
VegParams(fpath=inp_file,processes=['plantsize','dispersal','colonization','storage'])
#VegParams(processes=['plantsize','dispersal','colonization','storage','mortality'])
vegparams=load_params('veg_params.yml')
print(vegparams)


{'BIS': {'growparams': {'glucose_requirement': [1.444, 1.513, 1.463], 'growing_season_end': 305, 'growing_season_start': 45, 'k_light_extinct': 0.02, 'light_half_sat': 9, 'p_max': 0.055, 'plant_part_min': [0.01, 0.1, 0.5], 'respiration_coefficient': [0.015, 0.015, 0.03], 'root_to_leaf_coeffs': [0.09, 0.889, -0.0254], 'root_to_stem_coeffs': [-0.097, 1.071, 0.0179], 'senescence_start': 228}, 'plant_factors': {'angio_gymno': 'angiosperm', 'annual_perennial': 'perennial', 'growth_form': 'shrub', 'monocot_dicot': 'monocot', 'ptype': 'C3', 'species': 'Burial intolerant stabilizer '}, 'sizeparams': {'max_height_stem': 1.0, 'max_n_stems': 3.0, 'max_plant_density': 34.0, 'total_cs_area_stems': 0.231}, 'storparams': {'wint_dieoff_roots': 0.25}}, 'BTS': {'growparams': {'glucose_requirement': [1.444, 1.513, 1.463], 'growing_season_end': 305, 'growing_season_start': 45, 'k_light_extinct': 0.02, 'light_half_sat': 9, 'p_max': 0.055, 'plant_part_min': [0.01, 0.1, 0.5], 'respiration_coefficient': [0.01

### Set up model grid using landlab RasterModelGrid functions

- Create the model grid
- Initialize live vegetation biomass and plant functional type
- Initialize the air temperature driver across the grid

##### **Note variable names should be consistent with landlab standards

In [5]:
#Create grid 5x5 node grid with 2 m node spacing and xy of origin in lat long roughly corresponding to Island Beach State Park
pg = RasterModelGrid((6, 6), 2.0, xy_of_reference=(-74.08,39.79))
pg.axis_units = ('m', 'm')


                
#Initialize with a random mean air temperture 
_ = pg.add_field('air__temperature_C', np.random.uniform(low=12, high=14,size=pg.number_of_cells), at='cell', units='C' )
_ = pg.add_field('radiation__net_flux', np.random.uniform(low=0, high=180, size=pg.number_of_cells), at='cell', units='W/m^2')


Now we initialize plants. We can do this in multiple ways. Here we are going to seed two species randomly into grid cells and then count how many plants are in each cell to initialize the new fields.

In [6]:
#Initialize plant functional type as burial tolerant stabilizer or dune builder and randomly place on grid
plant_species_init=['BTS','DB']
plants_on_grid=[]
for ind,species in enumerate(plant_species_init):
    new_plants=np.random.choice(['null',species],pg.number_of_cells)
    if ind == 0:
        plants_on_grid=new_plants
    else:
        plants_on_grid=np.stack((plants_on_grid,new_plants),axis=1)

_ = pg.add_field('vegetation__plant_species',plants_on_grid, at='cell', clobber=True)

#occupied_ids=np.where(pg.at_cell['vegetation__plant_species']=='BTS')

pidval=0
plants=pd.DataFrame(columns=['pid','species','cell_index'])
for cell in range(pg.number_of_cells):
    cell_index=cell
    cell_plants=pg['cell']['vegetation__plant_species'][cell]
    for plant in cell_plants:
        species=plant
        newrow=[pidval,species,cell_index]
        plants.loc[pidval]=newrow
        pidval += 1

pid=plants.pid.loc[plants['species']=='BTS'].to_numpy(int)
total_biomass=np.random.rand(pid.shape[0])
total_biomass2=np.random.rand(6)

species_set=np.vstack((pid,total_biomass))
species_set2=np.vstack((total_biomass2,total_biomass2))
species_set=np.transpose(species_set)
species_set2=np.transpose(species_set2)
biomass=np.concatenate((species_set,species_set2), axis=0)
print(biomass)
biomass=np.transpose(biomass)
print(biomass)

In [7]:
#This was a check to see if I could save a list as a field and it is possible so we can have multiple species per cell
#veg_biomass=np.where(pg.at_cell['vegetation__plant_species'] != 'null',1,0)*np.random.uniform(low=0, high=200,size=pg.number_of_cells)
#veg_biomass_part=veg_biomass/3
#veg_biomass_tot=[]
#for ind, item in enumerate(veg_biomass):
#    veg_biomass_tot.append([item, veg_biomass_part[ind],veg_biomass_part[ind],veg_biomass_part[ind]])
#print(veg_biomass_tot)

### Plot grid with plant distribution and initial biomass

Use RasterModelGrid plot functions to visualize initial model grid

#### Vegetation type 
- 0=Bare
- 1=Annual
- 2=Herbaceous Perennial 
- 3=Woody Perennial

In [8]:
#Need to pudate to use colormap based on unique strings
#imshow_grid(pg, 'vegetation__plant_species', at='cell',colorbar_label='Vegetation Type')
#show()

In [9]:
#imshow_grid(pg, 'vegetation__live_biomass', at='cell', colorbar_label='Live biomass g')
#show()

### Define model time variables
Determine basic model timestep and counter for model loop

sim_end = date(sim_start_date.year+tot_sim_len_y,sim_start_date.month,sim_start_date.day)
dt = min([veg_time_step_d,env_time_step_d])
#data['dt']=dt
day_freq = str(dt)+'D'
date_counter = pd.date_range(start=sim_start_date,end=sim_end,freq=day_freq).strftime('%Y-%m-%d').tolist()
step_counter=range(len(date_counter))
model_counter=tuple(zip(step_counter,date_counter))

Initialize crops PlantGrowth class with veg_params from Excel file

In [10]:
croprun=GenVeg(pg,dt,sim_start_date,vegparams)
print(croprun.plant_ID())


    pid  cell_index species  leaf_biomass  stem_biomass  root_biomass
0     0           0     BTS      0.375534      0.235008      0.331235
1     1           2      DB      0.072285      0.037385      0.058568
2     2           3      DB      0.143635      0.079879      0.120570
3     3           4     BTS      0.336990      0.207902      0.295583
4     4           4      DB      0.069746      0.035945      0.056407
5     5           5     BTS      0.269379      0.161487      0.233570
6     6           5      DB      0.356669      0.221686      0.313761
7     7           6     BTS      0.083542      0.043837      0.068196
8     8           6      DB      0.348682      0.216078      0.306377
9     9           7      DB      0.081394      0.042598      0.066353
10   10           9     BTS      0.237891      0.140419      0.204951
11   11           9      DB      0.129594      0.071247      0.108209
12   12          10      DB      0.089558      0.047330      0.073369
13   13          11 

## Main model loop script
This is the main model loop that controls the simulation that
- Advances model time 
- Updates environmental variables
- Simulates changes to vegetation communities
- Stores outputs

In [11]:
#This runs through fine for day 1 of growing season. We need to decide how to track current day
for i in range(run_length_d):
    #update temp attribute on grid

    #update solar radiation on grid
    
    croprun.run_one_step(dt)
    print(i)
    #if this is a save timestep, save grid object to netcdf
    #create record method in croprun that uses datarecord to save PlantGrowth object and grid at time
    #if i in savetime:
    #    croprun.record()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258


c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply

259
260
261
262
263
264
265
266
267
268
269
270
271
272
273


c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply

274
275
276
277
278
279
280
281
282
283
284
285
286
287
288

c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply


289
290
291
292
293
294
295
296
297
298
299
300
301
302


c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)
c:\Users\RDEL1CDP\Anaconda3\envs\landlab\lib\site-packages\numpy\ma\core.py:1013: RuntimeWarning: overflow encountered in multiply

303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364


In [14]:
display(croprun.plant_df())

,pid,cell_index,species,leaf_biomass,stem_biomass,root_biomass
0,0,0,BTS,0.375534,0.235008,0.331235
1,1,2,DB,0.072285,0.037385,0.058568
2,2,3,DB,0.143635,0.079879,0.120570
3,3,4,BTS,0.336990,0.207902,0.295583
4,4,4,DB,0.069746,0.035945,0.056407
5,5,5,BTS,0.269379,0.161487,0.233570
6,6,5,DB,0.356669,0.221686,0.313761
7,7,6,BTS,0.083542,0.043837,0.068196
8,8,6,DB,0.348682,0.216078,0.306377
9,9,7,DB,0.081394,0.042598,0.066353


In [13]:
temp_dir = tempfile.mkdtemp()
os.chdir(temp_dir)
write_netcdf("test.nc", pg, format="NETCDF3_64BIT", time=step_counter[0])

NameError: name 'step_counter' is not defined